# Create an unstructured data pipeline for GenAI retrievers

Before building vector search indexes, it's essential to first prepare your unstructured data through a dedicated data engineering step. This involves ingesting, cleaning, and transforming raw documents into a structured format—typically chunked, metadata-enriched, and stored in Delta tables. This foundation ensures your GenAI retrievers operate on high-quality, queryable content.

In [0]:
%run ../00_setup/config

In [0]:
query = f"""
SELECT path
FROM READ_FILES('/Volumes/{catalog_name}/{schema_name}/pdfs/', format => 'binaryFile')
LIMIT 2
"""

spark.sql(query).show(truncate=False)

In [0]:
dbutils.widgets.text("catalog_name", catalog_name)
dbutils.widgets.text("schema_name", schema_name)

In [0]:
%sql
SELECT path
FROM READ_FILES('/Volumes/' || :catalog_name || '/' || :schema_name || '/pdfs/', format => 'binaryFile')
LIMIT 2


## 1. Extracting the information using `ai_parse_document`
Databricks provides a builtin `ai_parse_document` function, leveraging AI to analyze and extract PDF information as text. This makes it super easy to ingest unstructured information!

- Easy to adopt. Recommended for the POC and long-term. 
- This AI function can work with PDF, JPG, and PNG. The product team recently added PPTX as well. 

https://docs.databricks.com/aws/en/sql/language-manual/functions/ai_parse_document?language=SQL

In [0]:
%sql
SELECT path FROM READ_FILES('/Volumes/databricks_workshop/jywu/pdfs/', format => 'binaryFile') LIMIT 2

In [0]:
%sql
SELECT ai_parse_document(content) AS parsed_document
FROM READ_FILES('/Volumes/databricks_workshop/jywu/pdfs/', format => 'binaryFile') LIMIT 2

In [0]:
%sql
WITH corpus AS (
  SELECT
    path,
    ai_parse_document(content) AS parsed
  FROM
    READ_FILES('/Volumes/databricks_workshop/jywu/pdfs/', format => 'binaryFile')
)
SELECT
  path,
  parsed:document:pages,
  parsed:document:elements,
  parsed:corrupted_data,
  parsed:error_status,
  parsed:metadata
FROM corpus;

In [0]:
%sql
-- unflatten json
WITH corpus AS (
  SELECT
    path,
    ai_parse_document(content) AS parsed
  FROM READ_FILES('/Volumes/databricks_workshops/jywu/pdfs/', format => 'binaryFile')
)
SELECT
  c.path                      AS doc_uri,
  e.col.bbox[0].page_id       AS page_id,
  e.col.content,
  e.col.description,
  e.col.type,
  e.col.bbox[0].coord[0]      AS x0,
  e.col.bbox[0].coord[1]      AS y0,
  e.col.bbox[0].coord[2]      AS x1,
  e.col.bbox[0].coord[3]      AS y1
FROM corpus c
LATERAL VIEW explode(
  from_json(
    variant_get(
      variant_get(c.parsed, '$.document', 'VARIANT'),
      '$.elements',
      'STRING'
    ),
    'ARRAY<STRUCT<
        page_id INT,
        content STRING,
        description STRING,
        type STRING,
        bbox ARRAY<STRUCT<
          page_id INT,
          coord ARRAY<DOUBLE>
        >>
    >>'
  )
) e
LIMIT 20


In [0]:
%sql
 -- combine them together for chunking
WITH corpus AS (
  SELECT
    path,
    ai_parse_document(content) AS parsed
  FROM READ_FILES('/Volumes/databricks_workshop/jywu/pdfs/', format => 'binaryFile')
)
SELECT
  path AS doc_uri,
  array_join(
    transform(
      filter(
        parsed:document:elements::ARRAY<STRUCT<content:STRING, description:STRING>>,
        e -> e.content IS NOT NULL OR e.description IS NOT NULL
      ),
      e -> coalesce(e.content, e.description)
    ),
    '\n'
  ) AS content
FROM corpus;


## 2. Python UDF
Databricks' notebook [template](https://docs.databricks.com/aws/en/notebooks/source/generative-ai/unstructured-data-pipeline.html) to parse, chunk and create index. PDF, DOCX, HTML, TXT, MD, JSON are included in the template and the logic can be extended.
- Recommended for quick trial and error as it requires additional code.
- This is a more hard-coded way but more robust as well. 
- Search for `file_parser` function to take a closer look.